In [134]:
import numpy as np
import pandas as pd
import os

import os



## **중요**: ord averaging, hrtool은 나중에 보충해야됨 현재 포함 안되어있음

In [155]:
df = pd.read_csv("/Users/seanjoo/Desktop/horse/crawling/3차/경주성적정보_2019.csv")
df.head()
df['rcNo']

/var/folders/zf/4s7q2ydx00n7c1n6nz_8l2h40000gn/T/ipykernel_37316/3314225825.py:1: DtypeWarning: Columns (48) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/Users/seanjoo/Desktop/horse/crawling/3차/경주성적정보_2019.csv")


0        1
1        1
2        1
3        1
4        1
        ..
29187    9
29188    9
29189    9
29190    9
29191    9
Name: rcNo, Length: 29192, dtype: int64

In [136]:
# Selecting Only important conditions
df= df[["ilsu","hrName", "jeG1fTime", "ordBigo","rcDist", "rank", "budam", "age", "birthday", "chulNo", "diffUnit", "hrNo", "hrTool", "jkNo", "meet", "name", "ord", "owName", "rating", "rcDate", "rcTime", "sex", "trName", "wgBudam", "wgBudamBigo", "wgHr", "wgJk", "ageCond", "weather", "track", "rcName", "seS1fAccTime", "se_1cAccTime", "se_2cAccTime", "se_3cAccTime", "se_4cAccTime", "sjG1fOrd", "sjG3fOrd", "sjS1fOrd", "sj_1cOrd", "sj_2cOrd", "sj_3cOrd", "sj_4cOrd","rcNo"]
]

# Selecting only Seoul + 출장정지 없는거
df = df[(df['sjG1fOrd'] > 0) & (df['ordBigo'] == '-') & (df['jeG1fTime'] == 0)]
df = df.drop(columns=["ordBigo", "jeG1fTime"])

#Adjusting after thoughts
df = df.drop(columns = ["ilsu", "hrNo", "jkNo", "meet", "name","owName", "trName", "wgBudamBigo"])


In [137]:
#replace_dict = {
#    '국6등급': 6,
#    '국5등급': 5,
#    '국4등급': 4,
#    '혼4등급': 4,
#    '국3등급': 3, 
#    '혼3등급': 3,
#    '2등급': 2,
#    '1등급': 1,
#    '국OPEN': 7,  
#    '혼OPEN': 7, 
#}
#
#df['rank'] = df['rank'].replace(replace_dict)

In [138]:
calculate = df[['rcDate','hrName','ord']]
calculate = calculate.sort_values(by=['hrName', 'rcDate'])
calculate.head(20)


,rcDate,hrName,ord
16469,20190512,[부산경남]갑오데이,14
15748,20190602,[부산경남]굿캐스팅,8
15987,20190526,[부산경남]그레이트킹,12
9593,20191208,[부산경남]그레이트킹,7
16467,20190512,[부산경남]글리터,12
15330,20190616,[부산경남]글리터,8
15759,20190602,[부산경남]끝판에어로,3
15979,20190526,[부산경남]뉴레전드,4
12729,20190908,[부산경남]뉴레전드,9
10898,20191103,[부산경남]뉴레전드,1


In [139]:
def rankCal(x): 
    if len(x) < 2:
        return 0
    else:
         if (x <= 5).sum() > 2: 
             return 1 
         else: 
             return 0


In [140]:
calculate['winRate'] = calculate.groupby('hrName')['ord'].rolling(window=3, min_periods=2).apply(rankCal).reset_index(level=0, drop=True).fillna(value= 0)

In [141]:
len(calculate[calculate['winRate']==1])
calculate[calculate['winRate'] > 0]


,rcDate,hrName,ord,winRate
12707,20190908,가온챔프,3,1.0
11803,20191012,가온퀸,5,1.0
18686,20190310,가온포스,3,1.0
10792,20191109,가온포스,1,1.0
9182,20191221,가온포스,5,1.0
...,...,...,...,...
15598,20190608,희망스타,4,1.0
14717,20190706,희망스타,3,1.0
13807,20190810,희망스타,5,1.0
12901,20190901,히트더페이스,4,1.0


In [142]:
df = pd.merge(df, calculate, on=['hrName', 'rcDate'], suffixes=('_df', '_calc'))

df['winRate'] = df['winRate'].astype(int)


In [143]:
df = df.drop(columns = ['seS1fAccTime', 'se_1cAccTime', 'se_2cAccTime',
       'se_3cAccTime', 'se_4cAccTime','ord_calc',"hrTool",'sjG1fOrd', 'sjG3fOrd', 'sjS1fOrd', 'sj_1cOrd', 'sj_2cOrd', 'sj_3cOrd',
       'sj_4cOrd'])
df = df.rename(columns = {
    'ord_df' : 'ord'
})


In [144]:
df.columns

Index(['hrName', 'rcDist', 'rank', 'budam', 'age', 'birthday', 'chulNo',
       'diffUnit', 'ord', 'rating', 'rcDate', 'rcTime', 'sex', 'wgBudam',
       'wgHr', 'wgJk', 'ageCond', 'weather', 'track', 'rcName', 'rcNo',
       'winRate'],
      dtype='object')

In [145]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Encode categorical variables
label_encoders = {}
for column in ['hrName', 'rcDist', 'rank', 'budam','birthday', 
       'diffUnit',  'sex', 'ageCond', 'weather', 'track', 'rcName','wgHr']:  # Add all categorical columns
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

## Seperation of Data before pivot

In [146]:
half = df[['rcDate', 'rcNo',"rcDist","rank","budam","ageCond","weather","track","rcName"]]
half = half.groupby(by = ['rcDate', 'rcNo'], as_index= False).max()
half

,rcDate,rcNo,rcDist,rank,budam,ageCond,weather,track,rcName
0,20190105,1,0,5,1,5,1,0,73
1,20190105,2,0,5,1,5,1,0,73
2,20190105,3,2,8,4,5,1,0,73
3,20190105,4,0,5,1,1,1,0,73
4,20190105,5,0,5,1,1,1,0,73
...,...,...,...,...,...,...,...,...,...
1096,20191222,11,5,3,4,5,1,8,73
1097,20191222,12,6,0,4,5,1,7,26
1098,20191222,13,5,8,2,5,1,7,73
1099,20191222,14,3,7,4,5,1,7,61


In [147]:
df = df.drop(columns = ["rcDist","rank","budam","ageCond","weather","track","rcName","hrName"])
df.head()

,age,birthday,chulNo,diffUnit,ord,rating,rcDate,rcTime,sex,wgBudam,wgHr,wgJk,rcNo,winRate
0,2,631,9,0,1,0,20191222,61.2,2,53.0,1939,0,1,0
1,2,607,12,27,2,0,20191222,61.7,0,55.0,1500,0,1,0
2,2,543,5,17,3,0,20191222,62.0,2,53.0,1642,0,1,0
3,2,557,8,1,4,0,20191222,62.2,1,55.0,378,0,1,0
4,2,587,6,16,5,0,20191222,62.5,0,55.0,1577,0,1,0


In [148]:
pivot_data = df.set_index(['rcDate', 'rcNo', 'chulNo']).unstack(level='chulNo')

# Create new column labels
new_columns = []
for col_info in pivot_data.columns.tolist():
    new_columns.append(f"{col_info[1]}_{col_info[0]}")  # Format: position_columnname

pivot_data.columns = new_columns

# Reset the index to make rcDate and rcNo as regular columns
pivot_data.reset_index(inplace=True)

In [149]:
final_dataset = pd.merge(half, pivot_data, on=['rcDate', 'rcNo'], suffixes=('_half', '_pivotData'))
final_dataset.columns

Index(['rcDate', 'rcNo', 'rcDist', 'rank', 'budam', 'ageCond', 'weather',
       'track', 'rcName', '1_age',
       ...
       '7_winRate', '8_winRate', '9_winRate', '10_winRate', '11_winRate',
       '12_winRate', '13_winRate', '14_winRate', '15_winRate', '16_winRate'],
      dtype='object', length=185)

In [150]:
final_dataset.to_csv("/Users/seanjoo/Desktop/horse/오 시팔.csv") 